# 问题定义
1. 明确系统要解决的问题
2. 明确训练监督类型：监督学习、无监督学习、半监督学习、自监督学习、强化学习。
3. 确定性能指标
   1. 平均方根误差(RMSE) - L2范数(欧几里德的范数)
   2. 平均绝对误差(MAE) - L1范数(曼哈顿范数)

# 准备数据
## 收集数据
根据要解决的问题，收集整理线性的数据
## 数据洞察
了解数据结构、类型和分布情况

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

HOUSING_DATA_SET = "./datasets/housing/housing.csv"

housing = pd.read_csv(HOUSING_DATA_SET)
housing.info()
housing.describe()
housing.hist(bins=50, figsize=(12,8))

# 划分测试集
需要提前划出数据的验证集(测试集)，防止出现数据窥探偏误(data snooping bias)
> 大脑是一个惊人的模式检测系统，这也意味着它极易过拟合：如果你查看测试集，你可能会偶然发现测试数据中一些看似有趣的模式，从而引导你选择一种特殊的机器学习模型。当你使用测试集估计泛化误差时，你的估计会过于乐观，并且你将启动一个性能不如预期的系统。这称为数据窥探偏差。
划分数据集是抽样过程，需要尽可能反应总体样本的分布情况。

## 随机抽样
抽样的数据集合足够大(相较于属性的数量)
**随机产生测试集**

    简单能工作，但是由于完全随机，每次产生的测试集都不同，理论上执行几次之后测试集所有内容会全部都出现在了训练集中
**采用唯一标识分割测试集**

    需要数据有唯一标识，慎用行索引(需要考虑中间插入/删除数据情况)
## 分层抽样
按照一个或多个属性的分布情况进行抽样，确保产生的测试集和完整集合在这些属性的分布上接近。


In [2]:


# 拆分数据集
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

# 准备训练数据

# 拆分数据和标签, 为什么要分割？如何关联起来？
train_set_data = train_set.drop("median_house_value", axis=1)
train_set_label = train_set["median_house_value"].copy()





# 准备训练集
做数据清理，处理文本和分类属性，处理特定的转换操作，特征缩放(归一化)

In [ ]:


# 处理空数据
train_set_data_num = train_set_data.drop("ocean_proximity", axis=1)

num_attrs = list(train_set_data_num)
print(num_attrs)
cat_attrs = ["ocean_proximity"]

num_pipline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler())
])

full_pipline = ColumnTransformer([
    ("num", num_pipline, num_attrs),
    ("cat", OneHotEncoder(), cat_attrs),
])
train_set_data_prepared = full_pipline.fit_transform(train_set_data)

# 模型训练和评估

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(train_set_data_prepared, train_set_label)

some_data = train_set_data.iloc[:5]
some_labels = train_set_label.iloc[:5]
some_data_prepared = full_pipline.transform(some_data)
print("Predctions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))